In [ ]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import matplotlib.pyplot as plt

DATASET_PATH = "mfcc_birdyy.json"

def load_data(dataset_path):
    
    with open(dataset_path, "r") as fp:
        data = json.load(fp)
        
    X = np.array(data["mfcc"])
    y = np.array(data["labels"])
    
    return X, y


def prepare_datasets(test_size, validation_size):
    
    X, y = load_data(DATASET_PATH)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size) 
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)
    
    return X_train, X_validation, X_test, y_train, y_validation, y_test
    
    
def build_model(input_shape):
    
    model = keras.Sequential() 

    # LSTM vrstvy
    model.add(keras.layers.LSTM(64, input_shape=input_shape, return_sequences=True))
    model.add(keras.layers.LSTM(64))
    
    # plne prepojená vrstva
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.3))
    
    
    # výstupná vrstva
    model.add(keras.layers.Dense(7, activation='softmax'))
    
    return model
    
    
def plot_history(history):
    fig, axs = plt.subplots(2, figsize=(10,7))
    
    axs[0].plot(history.history["accuracy"], label="train accuracy")
    axs[0].plot(history.history["val_accuracy"], label="test accuracy")
    axs[0].set_ylabel("Accuracy")
    axs[0].legend(loc="lower right")
    axs[0].set_title("Accuracy evaluation")

    
    axs[1].plot(history.history["loss"], label="train error")
    axs[1].plot(history.history["val_loss"], label="test error")
    axs[1].set_ylabel("Error")
    axs[1].set_xlabel("Epoch")
    axs[1].legend(loc="upper right")
    axs[1].set_title("Error evaluation")
    
    plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.8, 
                    hspace=0.4)
    plt.show()
    

X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(0.25, 0.2)


input_shape = (X_train.shape[1], X_train.shape[2])
model = build_model(input_shape)


optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, 
              loss="sparse_categorical_crossentropy",
             metrics=["accuracy"])

model.summary()

history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=30)

test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy on test set is {}".format(test_accuracy))

plot_history(history)

In [ ]:
model.save('birdyy_tensorflow.h5')
print('Model Saved!')